In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
%matplotlib inline

In [ ]:
!pwd

In [ ]:
with open('exit_data_final.csv') as f:
    df = pd.read_csv(f)
f.close()

df.head()

In [ ]:
df.info()


In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
for var in df.columns:
    print(var, df[var].unique())

1. Looking at the unique values it seems we need some features have NULLs instead of np.nan
2. We need to transform the age values in order to force the correct sort order
3. We also need to transform the type of a few features

In [ ]:
# replacing blanks with np.nan
for var in df.columns:
    df[var].replace(to_replace=' ', value=np.nan, inplace=True)

In [ ]:
df['age'] = df['age'].map({'20 or younger': '1) 20 or younger',
                         '21 - 25': '2) 21 - 25', '26 - 30': '3) 26 - 30',
                         '31 - 35': '4) 31 - 35', '36 - 40': '5) 36 - 40',
                         '41 - 45': '6) 41 - 45', '46 - 50': '7) 46 - 50',
                         '56 or older': '8) 56 or older'})

In [ ]:
# binning/discretization of years_w/the_company
bins = [0,4,9,14,19,24]
labels = ['0-4yrs', '5-9yrs', '10-14yrs', '15-19yrs', '20+yrs']
df['tenure'] = pd.cut(df['tenure'], bins = bins, labels=labels)

In [ ]:
df.head()

In [ ]:
# Converting feature types
likert_items = df[['promotional_opportunities', 'performance_recognized',
       'feedback_offered', 'coaching_offered', 'mgmt_clear_mission',
       'mgmt_support_me', 'mgmt_support_team', 'mgmt_clear_comm',
       'direct_mgmt_satisfaction', 'job_stimulating', 'initiative_encouraged',
       'skill_variety', 'knowledge_variety', 'task_variety', 'fair_salary',
       'teamwork', 'team_support', 'team_comm', 'team_culture',
       'job_train_satisfaction', 'personal_train_satisfaction', 'org_culture',
       'grievances_resolution', 'co-worker_interaction',
       'workplace_conditions', 'job_stress', 'work/life_balance']]

for col in likert_items:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('float64')

In [ ]:
df.info()

In [ ]:
# Calculating latent variables
df['employee_valued'] = np.nanmean(df[['promotional_opportunities',
                                            'performance_recognized',
                                            'feedback_offered',
                                            'coaching_offered']], axis=1)

df['mgmt_sati'] = np.nanmean(df[['mgmt_clear_mission','mgmt_support_me',
                                      'mgmt_support_team','mgmt_clear_comm',
                                      'direct_mgmt_satisfaction']], axis=1)

df['job_satisfaction'] = np.nanmean(df[['job_stimulating','initiative_encouraged',
                                     'skill_variety','knowledge_variety',
                                     'task_variety']], axis=1)

df['team_satisfaction'] = np.nanmean(df[['teamwork','team_support',
                                      'team_comm','team_culture']], axis=1)

df['training_satisfaction'] = np.nanmean(df[['job_train_satisfaction',
                                          'personal_train_satisfaction']], axis=1)

df['org_environment'] = np.nanmean(df[['org_culture','grievances_resolution',
                                  'co-worker_interaction','workplace_conditions']], axis=1)

df['work_life_balance'] = np.nanmean(df[['job_stress','work/life_balance']], axis=1)

df['overall_sati'] = np.nanmean(df[['promotional_opportunities', 'performance_recognized',
       'feedback_offered', 'coaching_offered', 'mgmt_clear_mission',
       'mgmt_support_me', 'mgmt_support_team', 'mgmt_clear_comm',
       'direct_mgmt_satisfaction', 'job_stimulating', 'initiative_encouraged',
       'skill_variety', 'knowledge_variety', 'task_variety', 'fair_salary',
       'teamwork', 'team_support', 'team_comm', 'team_culture',
       'job_train_satisfaction', 'personal_train_satisfaction', 'org_culture',
       'grievances_resolution', 'co-worker_interaction',
       'workplace_conditions', 'job_stress', 'work/life_balance']], axis=1)

In [ ]:
df.head()

In [ ]:
mappings = {1:'1) Dissatisfied', 2:'1) Dissatisfied', 3:'2) Neutral', 4:'3) Satisfied', 5:'3) Satisfied'}
likert = ['promotional_opportunities', 'performance_recognized',
       'feedback_offered', 'coaching_offered', 'mgmt_clear_mission',
       'mgmt_support_me', 'mgmt_support_team', 'mgmt_clear_comm',
       'direct_mgmt_satisfaction', 'job_stimulating', 'initiative_encouraged',
       'skill_variety', 'knowledge_variety', 'task_variety', 'fair_salary',
       'teamwork', 'team_support', 'team_comm', 'team_culture',
       'job_train_satisfaction', 'personal_train_satisfaction', 'org_culture',
       'grievances_resolution', 'co-worker_interaction',
       'workplace_conditions', 'job_stress', 'work/life_balance']

for col in likert:
    df[col+'_short'] = df[col].map(mappings)
    
df.head()    

# Exploratory Data Analysis
1. We need to consider the fact employee exit surveys are almost always answered on a voluntary basis.  Due to this confounding variable we need to take any insights gleamed from the data as "evidence" of organizational affairs instead of definitive "proof".  Employees might have been extremely happy or angry with the organization and their attitude will surely be representated in their answers.   
2. Furthermore, our dataset contain only 700 employees which is simply too few to make any resonable recommendations. 
3. Also, we need to consider that we don't know the overall size of the organization and 700 terminations over the course of 4 years might be a relatively small number.  In other words, this survey does not provide proof of turnover rates w/in the organization. 
4. Finally, these 700 terminations are only respondents to the exit survey and we need to be care not to consider these as the total of all terminations which occurred in the last 4 years.  700 might be only a small percentage of all terminations. 

**In other words, our analysis hopes to determine the level satisfaction on several organizational factors for those employees who RESPONDED to the survey.  We are should not generalize our results to the broader organization or all terminated employees, who have not responded to the survey.**

**With that said, let's dig in!**

## Univariate Analysis

In [ ]:
def uni_plots(feature, text):
    tmp_count = df[feature].dropna().value_counts().values
    tmp_percent = ((df[feature].dropna().value_counts()/len(df))*100).values
    df1 = pd.DataFrame({feature: df[feature].value_counts().index,
                        'Number of Employees': tmp_count,
                        'Percent of Employees': tmp_percent})
    
    f, ax = plt.subplots(figsize=(20,10))
    plt.title(text, fontsize=25, pad=30)
    plt.tick_params(axis='both', labelsize=15, pad=10)
    plt.xlabel(feature, fontsize=20)
    plt.xticks(size=18)
    plt.yticks(size=18)
    
    sns.set_color_codes('pastel')
    count = sns.barplot(x=feature, y='Number of Employees', color='b', data=df1, label='Number of Employees')
    for p in count.patches:
        count.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center',
                   xytext = (0, 9), 
                   textcoords = 'offset points', size = 20)
    
    sns.set_color_codes('muted')
    percent = sns.barplot(x=feature, y='Percent of Employees', color='b', data=df1, label='Percent of Employees')
    for i in percent.patches:
        percent.annotate(format(i.get_height(), '.1f'), 
                   (i.get_x() + i.get_width() / 2., i.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), size = 20,
                   textcoords = 'offset points')
    
    ax.set_ylabel('')
    ax.legend(ncol=2, loc="upper right", fontsize=15, frameon=True)
    sns.despine(left=False, bottom=False)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    plt.show()

In [ ]:
def bi_cat_plot(feature1, feature2):
    ax = pd.crosstab(df[feature1], df[feature2], normalize='index')*100
    ax1 = ax.plot(kind='barh', stacked=True, figsize=(25,15), fontsize=25)
    for i in ax1.patches:
        width, height = i.get_width(), i.get_height()
        x, y = i.get_xy() 
        ax1.text(x+width/2,
                 y+height/2,
                 '{:.0f} %'.format(width),
                 horizontalalignment='center',
                 verticalalignment='center',
                 size=25)
    
    plt.title('Percentage of Termination by {} and {}'.format(feature1, feature2), fontsize=30, pad=25)
    plt.ylabel(' ')
    plt.legend(prop={'size':20})

In [ ]:
# plotting demographics against each likert item
def bi_likert_plot(feature1, feature2):
    ax = pd.crosstab(df[feature1], df[feature2], normalize='index')*100
    ax1 = ax.plot(kind='barh', stacked=True,figsize=(25,15), fontsize=25)
    
    for i in ax1.patches:
        width, height = i.get_width(), i.get_height()
        x, y = i.get_xy() 
        ax1.text(x+width/2,
                 y+height/2,
                 '{:.0f} %'.format(width),
                 horizontalalignment='center',
                 verticalalignment='center',
                 size=25)
    plt.title('Respondents attitude of {} by {}'.format(feature2, feature1), fontsize=30, pad=25)
    plt.ylabel(' ')
    plt.legend(prop={'size':20})

In [ ]:
def bi_likert_vol_plot(feature1, feature2):
    tmp_df = df.loc[(df['reason_of_term']=='vol_term')]
    ax = pd.crosstab(tmp_df[feature1], tmp_df[feature2], normalize='index')*100
    ax1 = ax.plot(kind='barh', stacked=True,figsize=(25,15), fontsize=25)
    
    for i in ax1.patches:
        width, height = i.get_width(), i.get_height()
        x, y = i.get_xy() 
        ax1.text(x+width/2,
                 y+height/2,
                 '{:.0f} %'.format(width),
                 horizontalalignment='center',
                 verticalalignment='center',
                 size=25)
    plt.title('Respondents attitude of {} by {}'.format(feature2, feature1), fontsize=30, pad=25)
    plt.ylabel(' ')
    plt.legend(prop={'size':20})

In [ ]:
def bi_cat_plot(feature1, feature2):
    ax = pd.crosstab(df[feature1], df[feature2], normalize='index')*100
    ax1 = ax.plot(kind='barh', stacked=True, figsize=(25,15), fontsize=25)
    for i in ax1.patches:
        width, height = i.get_width(), i.get_height()
        x, y = i.get_xy() 
        ax1.text(x+width/2,
                 y+height/2,
                 '{:.0f} %'.format(width),
                 horizontalalignment='center',
                 verticalalignment='center',
                 size=25)
    
    plt.title('Percentage of Termination Reasons by {}'.format(feature1), fontsize=30, pad=25)
    plt.ylabel(' ')
    plt.legend(prop={'size':20})

In [ ]:
# plotting demographics against the overall satisfaction score calculated for each repondent
# this allows us to determine what is the overall sati score by demographics.
# 
def overall_plot(feature):
    ax = round(df.groupby(feature)['overall_sati'].mean(),2).sort_values().plot(kind='barh', stacked=True,
                                                              figsize=(25,15), fontsize=25)
    for i in ax.patches:
        width, height = i.get_width(), i.get_height()
        x, y = i.get_xy() 
        ax.text(x+width/2,
                y+height/2,
                '{:.2f}'.format(width),
                horizontalalignment='center',
                verticalalignment='center',
                size=25)
    plt.title('Overall Employee Sentiment by {}'.format(feature), fontsize=30, pad=25)
    plt.ylabel(' ')

In [ ]:
# plotting demographics against individually averaged likert items (ie. employee_valued)
def overall_plot2(feature1, feature2):
    ax = round(df.groupby(feature1)[feature2].mean(),2).sort_values().plot(kind='barh', stacked=True,
                                                              figsize=(25,15), fontsize=25)
    for i in ax.patches:
        width, height = i.get_width(), i.get_height()
        x, y = i.get_xy() 
        ax.text(x+width/2,
                y+height/2,
                '{:.2f}'.format(width),
                horizontalalignment='center',
                verticalalignment='center',
                size=25)
    plt.title('Overall Employee {} Sentiment by {}'.format(feature2, feature1), fontsize=30, pad=25)
    plt.ylabel(' ')

In [ ]:
# plotting average likert sentiment by respondent characteristics for voluntary terminations
def bi_volterm_plot(feature1, feature2):
    tmp_df = df.loc[(df['reason_of_term']=='vol_term')]
    ax = round(tmp_df.groupby(feature1)[feature2].mean(),2).sort_values().plot(kind='barh', stacked=True,
                                                              figsize=(25,15), fontsize=25)
    for i in ax.patches:
        width, height = i.get_width(), i.get_height()
        x, y = i.get_xy() 
        ax.text(x+width/2,
                 y+height/2,
                 '{:.2f}'.format(width),
                 horizontalalignment='center',
                 verticalalignment='center',
                 size=25)
    plt.title('Average {} Sentiment of Voluntary Terminations by {}'.format(feature2, feature1),fontsize=30, pad=25)
    plt.ylabel(' ')
    plt.legend(prop={'size':20})

In [ ]:
# Plotting overall sati for voluntary terms by respondent characteristics
def bi_volterm_overall_plot(feature):
    tmp_df = df.loc[(df['reason_of_term']=='vol_term')]
    ax = round(tmp_df.groupby(feature)['overall_sati'].mean(),2).sort_values().plot(kind='barh', stacked=True,
                                                              figsize=(25,15), fontsize=25)
    for i in ax.patches:
        width, height = i.get_width(), i.get_height()
        x, y = i.get_xy() 
        ax.text(x+width/2,
                 y+height/2,
                 '{:.2f}'.format(width),
                 horizontalalignment='center',
                 verticalalignment='center',
                 size=25)
    plt.title('Average Overall Sentiment of Voluntary Terminations by {}'.format(feature),fontsize=30, pad=25)
    plt.ylabel(' ')
    plt.legend(prop={'size':20})

## Respondents' Characteristics

In [ ]:
uni_plots('department', 'Count & Percent of Employees by Department')

In [ ]:
uni_plots('year_of_term', 'Count & Percent of Employees by Year of Termination')

In [ ]:
uni_plots('reason_of_term', 'Count & Percent of Employees by Reason_of_Term')

In [ ]:
bi_cat_plot('reason_of_term', 'tenure')

In [ ]:
bi_cat_plot('department', 'reason_of_term')

In [ ]:
uni_plots('age', 'Count & Percent of Employees by Age')

In [ ]:
bi_cat_plot('age', 'reason_of_term')

In [ ]:
uni_plots('gender', 'Count & Percent of Employees by Gender')

In [ ]:
bi_cat_plot('reason_of_term', 'gender')

In [ ]:

uni_plots('type_of_employee', 'Count & Percent of Employees by Type of Employee')

In [ ]:

uni_plots('job_type', 'Count & Percent of Employees by job_type')

In [ ]:
bi_cat_plot('job_type', 'reason_of_term')

In [ ]:
bi_cat_plot('job_type', 'tenure')

In [ ]:
uni_plots('tenure', 'Count & Percent of Employees by Tenure')

In [ ]:
bi_cat_plot('tenure', 'reason_of_term')

## Overall Repondents' Sentiment

In [ ]:
overall_plot('department')

In [ ]:
overall_plot('reason_of_term')

In [ ]:
overall_plot('gender')

In [ ]:
overall_plot('age')

In [ ]:
overall_plot('type_of_employee')

In [ ]:
overall_plot('job_type')

In [ ]:
overall_plot('tenure')

In [ ]:
emp_value_avg = round(np.mean(df['employee_valued']),2)
mgmt_sati_avg = round(np.mean(df['mgmt_sati']),2)
job_sati_avg = round(np.mean(df['job_satisfaction']),2)
team_sati_avg = round(np.mean(df['team_satisfaction']),2)
training_sati_avg = round(np.mean(df['training_satisfaction']),2)
fair_salary_avg = round(np.mean(df['fair_salary']),2)
org_env_avg = round(np.mean(df['org_environment']),2)
work_life_avg = round(np.mean(df['work_life_balance']),2)
overall_sati = round(np.mean([emp_value_avg, mgmt_sati_avg, job_sati_avg, team_sati_avg,
                            training_sati_avg, fair_salary_avg, org_env_avg, work_life_avg]), 2)
temp_dict = {'emp_value_avg': emp_value_avg, 'mgmt_sati_avg': mgmt_sati_avg,
             'job_sati_avg': job_sati_avg, 'team_sati_avg': team_sati_avg, 
             'training_sati_avg': training_sati_avg, 
             'fair_salary_avg': fair_salary_avg,
             'org_env_avg': org_env_avg,
             'work_life_avg': work_life_avg, 'overall_sati': overall_sati}
tmp_df = pd.DataFrame.from_dict(temp_dict, orient='index', columns=['average']).sort_values(by='average')

In [ ]:
vol_df = df.loc[(df['reason_of_term']=='vol_term')]

vol_emp_value_avg = round(np.mean(vol_df['employee_valued']),2)
vol_mgmt_sati_avg = round(np.mean(vol_df['mgmt_sati']),2)
vol_job_sati_avg = round(np.mean(vol_df['job_satisfaction']),2)
vol_team_sati_avg = round(np.mean(vol_df['team_satisfaction']),2)
vol_training_sati_avg = round(np.mean(vol_df['training_satisfaction']),2)
vol_fair_salary_avg = round(np.mean(vol_df['fair_salary']),2)
vol_org_env_avg = round(np.mean(vol_df['org_environment']),2)
vol_work_life_avg = round(np.mean(vol_df['work_life_balance']),2)
vol_overall_sati = round(np.mean([vol_emp_value_avg, vol_mgmt_sati_avg, vol_job_sati_avg, 
                                  vol_team_sati_avg, vol_fair_salary_avg,
                                  vol_training_sati_avg, 
                                  vol_org_env_avg, vol_work_life_avg]), 2)
vol_temp_dict = {'emp_value_avg': vol_emp_value_avg, 'mgmt_sati_avg': vol_mgmt_sati_avg,
                 'job_sati_avg': vol_job_sati_avg, 'team_sati_avg': vol_team_sati_avg, 
                 'training_sati_avg': vol_training_sati_avg,
                 'fair_salary_avg': vol_fair_salary_avg,
                 'org_env_avg': vol_org_env_avg,
                 'work_life_avg': vol_work_life_avg, 'overall_sati': vol_overall_sati}
vol_tmp_df = pd.DataFrame.from_dict(vol_temp_dict, orient='index', columns=['average']).sort_values(by='average')

In [ ]:
plt.figure(figsize=(25,15))
plt.title('Overall Latent Factor Averages for Voluntary Termination', fontsize=28)
plt.ylabel('Average Employee Rating', fontsize=25)
ax = vol_tmp_df['average'].plot(kind='barh', fontsize=25)
for i in ax.patches:
    width, height = i.get_width(), i.get_height()
    x, y = i.get_xy() 
    ax.text(x+width/2,
            y+height/2,
            '{:.2f}'.format(width),
            horizontalalignment='center',
            verticalalignment='center',
            size=25)

plt.grid(False)
plt.show()

In [ ]:
plt.figure(figsize=(25,15))
plt.title('Overall Latent Factor Averages', fontsize=28)
plt.ylabel('Average Employee Rating', fontsize=25)
ax = tmp_df['average'].plot(kind='barh', fontsize=25)
for i in ax.patches:
    width, height = i.get_width(), i.get_height()
    x, y = i.get_xy() 
    ax.text(x+width/2,
            y+height/2,
            '{:.2f}'.format(width),
            horizontalalignment='center',
            verticalalignment='center',
            size=25)

plt.grid(False)
plt.show()

In [ ]:
likert = ['promotional_opportunities', 'performance_recognized',
       'feedback_offered', 'coaching_offered', 'mgmt_clear_mission',
       'mgmt_support_me', 'mgmt_support_team', 'mgmt_clear_comm',
       'direct_mgmt_satisfaction', 'job_stimulating', 'initiative_encouraged',
       'skill_variety', 'knowledge_variety', 'task_variety', 'fair_salary',
       'teamwork', 'team_support', 'team_comm', 'team_culture',
       'job_train_satisfaction', 'personal_train_satisfaction', 'org_culture',
       'grievances_resolution', 'co-worker_interaction',
       'workplace_conditions', 'job_stress', 'work/life_balance']

likert_avgs = []
for col in likert:
    likert_avgs.append(round(np.nanmean(df[col]),2))

In [ ]:
likert_avgs_df = pd.DataFrame(list(zip(likert, likert_avgs)), columns=['likert_item', 'avg_sentiment'])
likert_avgs_df.set_index('likert_item', inplace=True)


In [ ]:
plt.figure()

ax = likert_avgs_df.plot(kind='bar', figsize=(25,15), fontsize=20)
for i in ax.patches:
        width, height = i.get_width(), i.get_height()
        x, y = i.get_xy() 
        ax.text(x+width-0.25,
                 y+height+.1,
                 '{:.2f}'.format(height),
                 horizontalalignment='center',
                 verticalalignment='center',
                 size=20)
plt.title('Average Overall Likert Item Sentiment', fontsize=30, pad=25)
plt.legend().remove()
plt.xlabel(' ')        

## Voluntary Termination Construct Sentiment

In [ ]:
bi_volterm_plot('department', 'employee_valued')

In [ ]:
bi_volterm_plot('department', 'promotional_opportunities')

In [ ]:
bi_volterm_plot('age', 'promotional_opportunities')

In [ ]:
bi_volterm_plot('job_type', 'promotional_opportunities')

In [ ]:
bi_volterm_plot('tenure', 'promotional_opportunities')

In [ ]:
bi_volterm_plot('department', 'performance_recognized')

In [ ]:
bi_volterm_plot('age', 'performance_recognized')

In [ ]:
bi_volterm_plot('job_type', 'performance_recognized')

In [ ]:
bi_volterm_plot('tenure', 'performance_recognized')

## Org_env

In [ ]:
bi_volterm_plot('department', 'org_environment')

In [ ]:
bi_volterm_plot('department', 'org_culture')

In [ ]:
bi_volterm_plot('age', 'org_culture')

In [ ]:
bi_volterm_plot('job_type', 'org_culture')

In [ ]:
bi_volterm_plot('tenure', 'org_culture')

In [ ]:
bi_volterm_plot('department', 'grievances_resolution')

In [ ]:
bi_volterm_plot('age', 'grievances_resolution')

In [ ]:
bi_volterm_plot('job_type', 'grievances_resolution')

In [ ]:
bi_volterm_plot('tenure', 'grievances_resolution')

### Fair_salary

In [ ]:
bi_volterm_plot('department', 'fair_salary')

In [ ]:
bi_volterm_plot('age', 'fair_salary')

In [ ]:
bi_volterm_plot('job_type', 'fair_salary')

In [ ]:
bi_volterm_plot('tenure', 'fair_salary')

# Additional analysis not featured in medium article. 

### Year of Term

In [ ]:
def line_plot(feature):
    plt.figure(figsize=(15,10))
    sns.lineplot(x='year_of_term', y=feature, data=df, ci=None)
    plt.title('Respondents attitude of {} by Year of Termination'.format(feature), fontsize=20, pad=25)
    plt.ylabel(' ')

In [ ]:
line_plot('employee_valued')

In [ ]:
demos = ['department', 'location','year_of_term','reason_of_term','gender',
       'age', 'type_of_employee', 'job_type','tenure']

likert_short = ['promotional_opportunities_short', 'performance_recognized_short',
       'feedback_offered_short', 'coaching_offered_short',
       'mgmt_clear_mission_short', 'mgmt_support_me_short',
       'mgmt_support_team_short', 'mgmt_clear_comm_short',
       'direct_mgmt_satisfaction_short', 'job_stimulating_short',
       'initiative_encouraged_short', 'skill_variety_short',
       'knowledge_variety_short', 'task_variety_short', 'fair_salary_short',
       'teamwork_short', 'team_support_short', 'team_comm_short',
       'team_culture_short', 'job_train_satisfaction_short',
       'personal_train_satisfaction_short', 'org_culture_short',
       'grievances_resolution_short', 'co-worker_interaction_short',
       'workplace_conditions_short', 'job_stress_short',
       'work/life_balance_short']
for col in demos:
    for var in likert_short:
        bi_likert_plot(col, var)

In [ ]:
df_corr = df.corr()

plt.figure(figsize=(25,15))
mask = np.zeros_like(df_corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
plt.figure(figsize=(70,40))
plt.xticks(fontsize=50)
plt.yticks(fontsize=50)
sns.heatmap(df_corr, cmap='coolwarm', annot=True,
            fmt=".1f", annot_kws={'size': 40, 'color': 'black'}, linewidths=2,
            vmin=-0.5, mask=mask)



### Dealing with Missing Values

In [ ]:
tol_count = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/len(df))*100
tmp = pd.concat([tol_count, percent], axis=1, keys=['total_count','percent'])

f, ax = plt.subplots(figsize=(15, 5))
plt.title('Percentage of Missing Data')

sns.set_color_codes("pastel")
sns.barplot(x=tmp.index, y=tmp['total_count'], color='b', label='total count')

sns.set_color_codes("muted")
sns.barplot(x=tmp.index, y=tmp['percent'], color='b', label='percentage')

ax.legend(ncol=2, loc="upper right", frameon=True)
sns.despine(left=True, bottom=True)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.show()

We don't have any features warrent deletion due to a large number of missing values

In [ ]:
# drop any rows which have less than 28 non-null values
df.dropna(thresh=28, inplace=True)

In [ ]:
tol_count = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/len(df))*100
tmp = pd.concat([tol_count, percent], axis=1, keys=['total_count','percent'])

f, ax = plt.subplots(figsize=(15, 5))
plt.title('Percentage of Missing Data')

sns.set_color_codes("pastel")
sns.barplot(x=tmp.index, y=tmp['total_count'], color='b', label='total count')

sns.set_color_codes("muted")
sns.barplot(x=tmp.index, y=tmp['percent'], color='b', label='percentage')

ax.legend(ncol=2, loc="upper right", frameon=True)
sns.despine(left=True, bottom=True)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.show()

In [ ]:
cat_features = ['department', 'location', 'marital_status', 'reason_of_term', 'gender',
               'age', 'type_of_employee', 'job_type']

for col in cat_features:
    if (df[col].isnull().sum() / len(df)) > 0.1:
        df[col].fillna('missing')
    else:
        df[col].fillna(df.mode())

In [ ]:
df.location.isnull().sum() / len(df)

In [ ]:
tol_count = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/len(df))*100
tmp = pd.concat([tol_count, percent], axis=1, keys=['total_count','percent'])

f, ax = plt.subplots(figsize=(15, 5))
plt.title('Percentage of Missing Data')

sns.set_color_codes("pastel")
sns.barplot(x=tmp.index, y=tmp['total_count'], color='b', label='total count')

sns.set_color_codes("muted")
sns.barplot(x=tmp.index, y=tmp['percent'], color='b', label='percentage')

ax.legend(ncol=2, loc="upper right", frameon=True)
sns.despine(left=True, bottom=True)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.show()